In [1]:
from pygam import LinearGAM, s, f

In [2]:
import dask.dataframe as ddf
import pandas as pd
import datetime
import functools
import seaborn as sns
import matplotlib.pyplot as plt

### Load income data

In [3]:
years = [year for year in range(2009, 2019)]
years

[2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018]

In [4]:
path = "/global/cfs/cdirs/m1532/Projects_MVP/geospatial/SHAP_Aggregated_0918/Social_Economic_Factors/"

In [5]:
data_full = []
data = []
for year in years:
    social_year = pd.read_csv(path + 'Social_Economic_' + str(year) + '.csv', dtype = {'fips': str, 'per_capita_income': float})
    social_year = social_year.loc[:, ~social_year.columns.str.contains('^Unnamed')]
    social_year['year'] = year
    income = social_year[['year', 'fips', 'per_capita_income']]
    data_full.append(social_year)
    data.append(income)

In [6]:
income_0918 = pd.concat(data)
income_0918

,year,fips,per_capita_income
0,2009,01001,23774.0
1,2009,01003,26197.0
2,2009,01005,15842.0
3,2009,01007,18953.0
4,2009,01009,20360.0
...,...,...,...
3215,2018,72145,10601.0
3216,2018,72147,11156.0
3217,2018,72149,10677.0
3218,2018,72151,9148.0


In [8]:
income_full = pd.concat(data_full)
income_full = income_full.drop('rurality', axis = 1)
income_full

,fips,per_capita_income,percent_below_poverty,percent_no_highschool_diploma,percent_mobile_homes,percent_no_vehicle,percent_unemployed_CDC,percent_overcrowding,percent_multi_unit_housing,percent_age_65_and_older,percent_limited_english_abilities,percent_minorities,percent_disabled,percent_institutionalized_in_group_quarters,percent_single_parent_households_CDC,percent_age_18_to_34,percent_age_35_to_44,percent_age_45_to_54,percent_age_55_to_64,year
0,01001,23774.0,10.306176,15.253106,19.822927,4.773622,NaN,1.252187,1.423398,11.013633,3.499199,22.079703,NaN,0.000000,NaN,20.361407,15.934576,14.436108,10.148435,2009
1,01003,26197.0,11.924937,12.808534,15.672453,3.451246,19.388429,1.918332,15.329419,16.670640,4.387162,14.893283,22.182166,1.024436,23.949134,19.088124,13.497328,14.728745,12.702547,2009
2,01005,15842.0,24.749648,30.609873,28.644396,12.221700,NaN,2.198215,0.918493,13.262313,4.156629,51.956983,NaN,9.577588,NaN,23.618649,14.607423,13.616290,11.971143,2009
3,01007,18953.0,11.896527,28.421561,32.640070,7.441860,NaN,1.873754,3.342522,12.854081,1.253841,24.608647,NaN,4.123183,NaN,20.671823,16.809542,14.596534,11.176854,2009
4,01009,20360.0,10.953932,27.435538,24.758860,3.817350,NaN,2.865613,0.592660,14.016619,6.673074,10.592564,NaN,0.896064,NaN,20.306669,14.886276,14.238434,12.034364,2009
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3215,72145,10601.0,45.298920,32.707645,0.201647,15.861955,40.055573,3.052928,1.333550,18.667441,89.898692,97.225085,40.429600,0.577092,32.101008,22.150606,12.420603,13.288115,12.829064,2018
3216,72147,11156.0,41.260745,25.246737,0.000000,24.979757,34.657879,5.465587,0.000000,20.818607,87.333977,94.322198,12.152962,0.000000,27.995521,19.724091,12.005473,12.279102,14.810170,2018
3217,72149,10677.0,48.156097,26.465201,0.236189,12.162863,40.856031,3.708506,3.509737,15.861349,94.047072,99.839081,46.403905,0.565389,41.454404,23.128778,11.486104,13.238812,13.738964,2018
3218,72151,9148.0,53.468225,31.368122,0.000000,17.232554,44.135980,2.653131,0.685396,18.565697,92.491509,99.956075,24.079629,0.213769,40.436921,21.192421,12.249261,13.602155,14.120472,2018


In [9]:
income_full.to_csv('social_economic_factors_0918.csv')

In [88]:
income_0918 = income_0918.sort_values(['fips', 'year'])
income_0918.head()

,year,fips,per_capita_income
0,2009,01001,23774.0
0,2010,01001,24568.0
0,2011,01001,25035.0
0,2012,01001,25253.0
0,2013,01001,24571.0


In [86]:
max_income = income_0918['per_capita_income'].max()
max_income

72832.0

In [87]:
min_income = income_0918['per_capita_income'].min()
min_income

5327.0

In [89]:
percent1 = (23774 - min_income)/(max_income-min_income)
percent1

0.27326864676690615

#### since we can't get income data before 2009 and after 2018 from censue.gov, we map 2009 income data to 2000-2008 and 2018 income data to 2019

In [7]:
years_before = [year for year in range(2000, 2009)]

In [8]:
years_before

[2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008]

In [9]:
income_2009 = income_0918[income_0918['year'] == 2009]
income_2019 = income_0918[income_0918['year'] == 2018]
income_2019['year'] = 2019
data_before = []
for year in years_before:
    income_year = income_2009[['fips', 'per_capita_income']]
    income_year['year'] = year
    data_before.append(income_year)

/tmp/ipykernel_96567/2320425100.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  income_2019['year'] = 2019


In [10]:
income_before = pd.concat(data_before)
income_before = income_before[['year', 'fips', 'per_capita_income']]
income_before.head()

,year,fips,per_capita_income
0,2000,01001,23774.0
1,2000,01003,26197.0
2,2000,01005,15842.0
3,2000,01007,18953.0
4,2000,01009,20360.0


In [11]:
data_final = [income_before, income_0918, income_2019]
income_final = pd.concat(data_final)
income_final

,year,fips,per_capita_income
0,2000,01001,23774.0
1,2000,01003,26197.0
2,2000,01005,15842.0
3,2000,01007,18953.0
4,2000,01009,20360.0
...,...,...,...
3215,2019,72145,10601.0
3216,2019,72147,11156.0
3217,2019,72149,10677.0
3218,2019,72151,9148.0


In [12]:
income_final = income_final.rename(columns = {'per_capita_income': 'income'})
income_final.head()

,year,fips,income
0,2000,01001,23774.0
1,2000,01003,26197.0
2,2000,01005,15842.0
3,2000,01007,18953.0
4,2000,01009,20360.0


In [19]:
income_median = income_final.drop(['fips'], axis = 1)
income_median = income_median.groupby(['year']).median()
income_median = income_median.reset_index()
income_median

,year,income
0,2000,21369.0
1,2001,21369.0
2,2002,21369.0
3,2003,21369.0
4,2004,21369.0
5,2005,21369.0
6,2006,21369.0
7,2007,21369.0
8,2008,21369.0
9,2009,21369.0


In [20]:
income_median = income_median.rename(columns = {'income': 'median_income'})
income_final = income_final.merge(income_median, on = ['year'], how = 'inner')
income_final.head()

,year,fips,income,median_income
0,2000,01001,23774.0,21369.0
1,2000,01003,26197.0,21369.0
2,2000,01005,15842.0,21369.0
3,2000,01007,18953.0,21369.0
4,2000,01009,20360.0,21369.0


In [23]:
def divideByMedianIncome(income, median_income):
    if income >= median_income:
        return 'above'
    else:
        return 'below'

In [24]:
income_final['income_group'] = income_final.apply(lambda x: divideByMedianIncome(x['income'], x['median_income']), axis = 1)
income_final.head()

,year,fips,income,median_income,income_group
0,2000,01001,23774.0,21369.0,above
1,2000,01003,26197.0,21369.0,above
2,2000,01005,15842.0,21369.0,below
3,2000,01007,18953.0,21369.0,below
4,2000,01009,20360.0,21369.0,below


### Import suicide death data

In [25]:
years = [year for year in range(2000, 2020)]
years

[2000,
 2001,
 2002,
 2003,
 2004,
 2005,
 2006,
 2007,
 2008,
 2009,
 2010,
 2011,
 2012,
 2013,
 2014,
 2015,
 2016,
 2017,
 2018,
 2019]

In [26]:
data = []
for year in years:
    suicide_year = pd.read_csv('/global/cfs/cdirs/m1532/Projects_MVP/geospatial/Suicide_Death_NCHS/suicide_patient_level_1960_2020/overall/overall_suicide_patient_level_' + str(year) + '.csv', dtype = {'year': int, 'county_residence': str, 'county_death': str, 'state_residence': str, 'state_death': str})
    suicide_year = suicide_year.loc[:, ~suicide_year.columns.str.contains('^Unnamed')]
    data.append(suicide_year)

In [27]:
suicide = pd.concat(data)
suicide.head()

,year,county_residence,state_residence,state_death,county_death,month,sex,race,age,age_range,death_cause
0,2000,01115,01,01,01055,Jan,Male,White,8,age 25-64,firearm_suicide
1,2000,01101,01,01,01101,Jan,Male,Black,4,age 0-24,firearm_suicide
2,2000,01001,01,01,01001,Jan,Female,White,6,age 25-64,firearm_suicide
3,2000,01003,01,01,01003,Jan,Female,White,3,age 0-24,nonfirearm_suicide
4,2000,01015,01,01,01015,Jan,Male,White,6,age 25-64,firearm_suicide


In [28]:
suicide = suicide.drop(['county_residence', 'state_residence'], axis = 1)
suicide = suicide.rename(columns = {'state_death': 'statefips', 'county_death': 'fips'})
suicide.head()

,year,statefips,fips,month,sex,race,age,age_range,death_cause
0,2000,01,01055,Jan,Male,White,8,age 25-64,firearm_suicide
1,2000,01,01101,Jan,Male,Black,4,age 0-24,firearm_suicide
2,2000,01,01001,Jan,Female,White,6,age 25-64,firearm_suicide
3,2000,01,01003,Jan,Female,White,3,age 0-24,nonfirearm_suicide
4,2000,01,01015,Jan,Male,White,6,age 25-64,firearm_suicide


In [29]:
suicide

,year,statefips,fips,month,sex,race,age,age_range,death_cause
0,2000,01,01055,Jan,Male,White,8,age 25-64,firearm_suicide
1,2000,01,01101,Jan,Male,Black,4,age 0-24,firearm_suicide
2,2000,01,01001,Jan,Female,White,6,age 25-64,firearm_suicide
3,2000,01,01003,Jan,Female,White,3,age 0-24,nonfirearm_suicide
4,2000,01,01015,Jan,Male,White,6,age 25-64,firearm_suicide
...,...,...,...,...,...,...,...,...,...
47669,2019,36,36081,Dec,Male,Other,5,age 25-64,nonfirearm_suicide
47670,2019,36,36081,Dec,Male,Black,5,age 25-64,nonfirearm_suicide
47671,2019,36,36005,Dec,Female,Black,4,age 0-24,nonfirearm_suicide
47672,2019,36,36005,Dec,Male,White,6,age 25-64,nonfirearm_suicide


### Load population data

In [30]:
data_pop = []
for year in years:
    pop_year = pd.read_csv('/global/cfs/cdirs/m1532/Projects_MVP/geospatial/temp_bins_suicide/Population/population_monthly/population_monthly_' + str(year) + '.csv', dtype = {'year': int, 'fips': str})
    pop_year = pop_year.loc[:, ~pop_year.columns.str.contains('^Unnamed')]
    data_pop.append(pop_year)

In [31]:
pop = pd.concat(data_pop)
pop.head()

,year,fips,pop,month
0,2000,01001,39646,Jan
1,2000,01001,39758,Feb
2,2000,01001,39870,Mar
3,2000,01001,39982,Apr
4,2000,01001,40094,May


In [32]:
month_replace = {'Jan': 1, 'Feb': 2, 'Mar': 3, 'Apr': 4, 'May': 5, 'Jun': 6, 'Jul': 7, 'Aug': 8, 'Sep': 9, 'Oct': 10,
                'Nov': 11, 'Dec': 12}
pop['month'] = pop['month'].replace(month_replace)
pop['month'].unique()

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12])

In [33]:
pop.head()

,year,fips,pop,month
0,2000,01001,39646,1
1,2000,01001,39758,2
2,2000,01001,39870,3
3,2000,01001,39982,4
4,2000,01001,40094,5


In [34]:
suicide

,year,statefips,fips,month,sex,race,age,age_range,death_cause
0,2000,01,01055,Jan,Male,White,8,age 25-64,firearm_suicide
1,2000,01,01101,Jan,Male,Black,4,age 0-24,firearm_suicide
2,2000,01,01001,Jan,Female,White,6,age 25-64,firearm_suicide
3,2000,01,01003,Jan,Female,White,3,age 0-24,nonfirearm_suicide
4,2000,01,01015,Jan,Male,White,6,age 25-64,firearm_suicide
...,...,...,...,...,...,...,...,...,...
47669,2019,36,36081,Dec,Male,Other,5,age 25-64,nonfirearm_suicide
47670,2019,36,36081,Dec,Male,Black,5,age 25-64,nonfirearm_suicide
47671,2019,36,36005,Dec,Female,Black,4,age 0-24,nonfirearm_suicide
47672,2019,36,36005,Dec,Male,White,6,age 25-64,nonfirearm_suicide


In [35]:
suicide_death = suicide.drop(['statefips', 'sex', 'race', 'age', 'age_range'], axis = 1)
suicide_death = suicide_death.groupby(['year', 'fips', 'month']).count()
suicide_death = suicide_death.reset_index()
suicide_death.head()

,year,fips,month,death_cause
0,2000,01001,Apr,1
1,2000,01001,Aug,2
2,2000,01001,Feb,2
3,2000,01001,Jan,1
4,2000,01001,Jun,1


In [36]:
suicide_death['year'].unique()

array([2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010,
       2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019])

In [37]:
suicide_death['month'] = suicide_death['month'].replace(month_replace)
suicide_death.head()

,year,fips,month,death_cause
0,2000,01001,4,1
1,2000,01001,8,2
2,2000,01001,2,2
3,2000,01001,1,1
4,2000,01001,6,1


In [38]:
suicide_pop = suicide_death.merge(pop, on = ['year', 'month', 'fips'], how = 'inner')
suicide_pop.head()

,year,fips,month,death_cause,pop
0,2000,01001,4,1,39982
1,2000,01001,8,2,40431
2,2000,01001,2,2,39758
3,2000,01001,1,1,39646
4,2000,01001,6,1,40207


In [39]:
suicide_pop = suicide_pop.rename(columns = {'death_cause': 'deaths'})
suicide_pop.head()

,year,fips,month,deaths,pop
0,2000,01001,4,1,39982
1,2000,01001,8,2,40431
2,2000,01001,2,2,39758
3,2000,01001,1,1,39646
4,2000,01001,6,1,40207


In [41]:
income_final = income_final[['year', 'fips', 'income_group']]
suicide_pop = suicide_pop.merge(income_final, on = ['year', 'fips'], how = 'inner')
suicide_pop

,year,fips,month,deaths,pop,income_group
0,2000,01001,4,1,39982,above
1,2000,01001,8,2,40431,above
2,2000,01001,2,2,39758,above
3,2000,01001,1,1,39646,above
4,2000,01001,6,1,40207,above
...,...,...,...,...,...,...
273478,2019,56041,1,1,17448,above
273479,2019,56041,7,1,18652,above
273480,2019,56041,5,1,18251,above
273481,2019,56041,10,1,19254,above


In [42]:
suicide_pop = suicide_pop.drop(['fips'], axis = 1)
suicide_pop = suicide_pop.groupby(['year', 'month', 'income_group']).sum()
suicide_pop = suicide_pop.reset_index()
suicide_pop.head()

,year,month,income_group,deaths,pop
0,2000,1,above,1992,168800585
1,2000,1,below,483,19498668
2,2000,2,above,1795,166025049
3,2000,2,below,462,19162905
4,2000,3,above,1937,165489299


### Load temperature and precipitation data from PRISM

In [58]:
data_temp = []
for year in years:
    temp_year = pd.read_csv('/global/cfs/cdirs/m1532/Projects_MVP/geospatial/PRISM_Data/PRISM_daily_county_level/prism_daily_county_level_' + str(year) + '.csv', dtype = {'year': int, 'fips': str})
    temp_year = temp_year.loc[:, ~temp_year.columns.str.contains('^Unnamed')]
    data_temp.append(temp_year)

In [59]:
climate = pd.concat(data_temp)
climate.head()

,fips,date,tMean,tMin,tMax,prec,year
0,01001,2000-01-01,16.262459,10.089380,22.435537,0.560423,2000
1,01001,2000-01-02,17.412651,11.954220,22.871081,0.390251,2000
2,01001,2000-01-03,18.705264,14.001039,23.409490,0.216170,2000
3,01001,2000-01-04,16.203456,9.861904,22.545009,16.488676,2000
4,01001,2000-01-05,6.494604,-1.153744,14.142953,0.216055,2000


In [60]:
def celsius_to_fahrenheit(celsius):
    return (celsius * 9/5) + 32

In [61]:
climate['tMean'] = climate['tMean'].apply(celsius_to_fahrenheit)
climate['tMin'] = climate['tMin'].apply(celsius_to_fahrenheit)
climate['tMax'] = climate['tMax'].apply(celsius_to_fahrenheit)

In [62]:
import datetime

In [63]:
def convertTime(time):
    return datetime.strptime(time, '%Y-%m-%d')

In [64]:
def getMonth(date):
    return(date.strftime('%b'))

In [65]:
climate['date'] = pd.to_datetime(climate['date'])
climate['month'] = climate['date'].apply(getMonth)

In [66]:
climate['month'] = climate['month'].replace(month_replace)
climate.head()

,fips,date,tMean,tMin,tMax,prec,year,month
0,01001,2000-01-01,61.272425,50.160884,72.383967,0.560423,2000,1
1,01001,2000-01-02,63.342771,53.517596,73.167946,0.390251,2000,1
2,01001,2000-01-03,65.669476,57.201870,74.137082,0.216170,2000,1
3,01001,2000-01-04,61.166221,49.751427,72.581016,16.488676,2000,1
4,01001,2000-01-05,43.690288,29.923261,57.457315,0.216055,2000,1


In [67]:
climate = climate.drop(['date'], axis = 1)
climate = climate.groupby(['year', 'month', 'fips']).mean()
climate = climate.reset_index()
climate.head()

,year,month,fips,tMean,tMin,tMax,prec
0,2000,1,01001,48.166560,37.347152,58.985968,4.190979
1,2000,1,01003,52.781099,41.531900,64.030298,2.639092
2,2000,1,01005,48.316217,36.439576,60.192859,3.507654
3,2000,1,01007,45.746169,34.330670,57.161669,4.497632
4,2000,1,01009,42.205221,32.078420,52.332022,4.782151


In [68]:
climate

,year,month,fips,tMean,tMin,tMax,prec
0,2000,1,01001,48.166560,37.347152,58.985968,4.190979
1,2000,1,01003,52.781099,41.531900,64.030298,2.639092
2,2000,1,01005,48.316217,36.439576,60.192859,3.507654
3,2000,1,01007,45.746169,34.330670,57.161669,4.497632
4,2000,1,01009,42.205221,32.078420,52.332022,4.782151
...,...,...,...,...,...,...,...
745195,2019,12,56037,17.121235,7.089279,27.153191,0.632654
745196,2019,12,56039,17.573244,9.812942,25.333545,2.434054
745197,2019,12,56041,18.615721,7.988787,29.242656,0.900065
745198,2019,12,56043,23.729262,12.816991,34.641533,0.354680


In [69]:
climate = climate.merge(income_final, on = ['year', 'fips'], how = 'inner')
climate.head()

,year,month,fips,tMean,tMin,tMax,prec,income_group
0,2000,1,01001,48.166560,37.347152,58.985968,4.190979,above
1,2000,2,01001,53.175360,39.102574,67.248145,1.577380,above
2,2000,3,01001,59.891998,46.674610,73.109385,3.348399,above
3,2000,4,01001,61.008822,47.667463,74.350182,2.889071,above
4,2000,5,01001,75.073873,62.554663,87.593083,1.175421,above


In [70]:
climate['income_group'].unique()

array(['above', 'below'], dtype=object)

In [71]:
climate = climate.drop(['fips'], axis = 1)
climate = climate.groupby(['year', 'month', 'income_group']).mean()
climate = climate.reset_index()
climate.head()

,year,month,income_group,tMean,tMin,tMax,prec
0,2000,1,above,30.497887,20.134824,40.860951,2.161118
1,2000,1,below,37.016266,25.951230,48.081303,2.269909
2,2000,2,above,37.652054,26.607609,48.696499,2.183954
3,2000,2,below,43.856360,31.367610,56.345110,2.144588
4,2000,3,above,45.846847,34.039125,57.654569,2.333613


In [72]:
climate

,year,month,income_group,tMean,tMin,tMax,prec
0,2000,1,above,30.497887,20.134824,40.860951,2.161118
1,2000,1,below,37.016266,25.951230,48.081303,2.269909
2,2000,2,above,37.652054,26.607609,48.696499,2.183954
3,2000,2,below,43.856360,31.367610,56.345110,2.144588
4,2000,3,above,45.846847,34.039125,57.654569,2.333613
...,...,...,...,...,...,...,...
475,2019,10,below,58.999602,46.890152,71.109052,3.502560
476,2019,11,above,38.958584,28.651418,49.265750,1.617319
477,2019,11,below,44.467513,32.592693,56.342333,2.024555
478,2019,12,above,35.052757,25.602590,44.502925,2.228846


In [73]:
suicide_pop.head()

,year,month,income_group,deaths,pop
0,2000,1,above,1992,168800585
1,2000,1,below,483,19498668
2,2000,2,above,1795,166025049
3,2000,2,below,462,19162905
4,2000,3,above,1937,165489299


In [74]:
suicide_climate = suicide_pop.merge(climate, on = ['year', 'month', 'income_group'], how = 'inner')
suicide_climate.head()

,year,month,income_group,deaths,pop,tMean,tMin,tMax,prec
0,2000,1,above,1992,168800585,30.497887,20.134824,40.860951,2.161118
1,2000,1,below,483,19498668,37.016266,25.951230,48.081303,2.269909
2,2000,2,above,1795,166025049,37.652054,26.607609,48.696499,2.183954
3,2000,2,below,462,19162905,43.856360,31.367610,56.345110,2.144588
4,2000,3,above,1937,165489299,45.846847,34.039125,57.654569,2.333613


In [75]:
suicide_climate['suicide_rate'] = (suicide_climate['deaths']/suicide_climate['pop'])*100000
suicide_climate.head()

,year,month,income_group,deaths,pop,tMean,tMin,tMax,prec,suicide_rate
0,2000,1,above,1992,168800585,30.497887,20.134824,40.860951,2.161118,1.180091
1,2000,1,below,483,19498668,37.016266,25.951230,48.081303,2.269909,2.477092
2,2000,2,above,1795,166025049,37.652054,26.607609,48.696499,2.183954,1.081162
3,2000,2,below,462,19162905,43.856360,31.367610,56.345110,2.144588,2.410908
4,2000,3,above,1937,165489299,45.846847,34.039125,57.654569,2.333613,1.170468


In [76]:
summer_winter = [6, 7, 8, 12, 1, 2]
def is_summer_winter(month):
    if month in summer_winter :
        return 1
    else:
        return 0

In [77]:
suicide_climate['summer_winter'] = suicide_climate['month'].apply(is_summer_winter)
suicide_climate.head()

,year,month,income_group,deaths,pop,tMean,tMin,tMax,prec,suicide_rate,summer_winter
0,2000,1,above,1992,168800585,30.497887,20.134824,40.860951,2.161118,1.180091,1
1,2000,1,below,483,19498668,37.016266,25.951230,48.081303,2.269909,2.477092,1
2,2000,2,above,1795,166025049,37.652054,26.607609,48.696499,2.183954,1.081162,1
3,2000,2,below,462,19162905,43.856360,31.367610,56.345110,2.144588,2.410908,1
4,2000,3,above,1937,165489299,45.846847,34.039125,57.654569,2.333613,1.170468,0


In [78]:
suicide_climate.to_csv('monthly_GAM_median_income.csv')